Credit: https://towardsdatascience.com/using-python-to-create-a-world-map-from-a-list-of-country-names-cd7480d03b10

In [1]:
import pandas as pd
import numpy as np
import igraph as ig
import collections
import matplotlib
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
import json
%matplotlib inline

In [2]:
df_leading_eigenvector = pd.read_csv("df_leading_eigenvector_no_inter.csv")
df_leading_eigenvector

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,0
2,2,Algeria,1
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
190,190,Venezuela,0
191,191,Vietnam,1
192,192,Yemen,0
193,193,Zambia,1


In [3]:
df_spinglass = pd.read_csv('df_spinglass_no_inter.csv')
df_spinglass

,Unnamed: 0,name,community
0,0,Albania,2
1,1,Bahamas,2
2,2,Algeria,4
3,3,British Virgin Islands,5
4,4,United Kingdom,4
...,...,...,...
190,190,Venezuela,1
191,191,Vietnam,5
192,192,Yemen,1
193,193,Zambia,5


In [4]:
df_walktrap = pd.read_csv('df_walktrap_no_inter.csv')
df_walktrap

,Unnamed: 0,name,community
0,0,Albania,0
1,1,Bahamas,1
2,2,Algeria,2
3,3,British Virgin Islands,1
4,4,United Kingdom,1
...,...,...,...
190,190,Venezuela,1
191,191,Vietnam,1
192,192,Yemen,63
193,193,Zambia,64


In [5]:
def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

In [6]:
df_spinglass['codes'] = df_spinglass['name'].apply(get_continent)
df_spinglass['country_code'] = df_spinglass['codes'].apply(lambda x: x[0])
df_spinglass['continent_code'] = df_spinglass['codes'].apply(lambda x: x[1])

df_walktrap['codes'] = df_walktrap['name'].apply(get_continent)
df_walktrap['country_code'] = df_walktrap['codes'].apply(lambda x: x[0])
df_walktrap['continent_code'] = df_walktrap['codes'].apply(lambda x: x[1])

df_leading_eigenvector['codes'] = df_leading_eigenvector['name'].apply(get_continent)
df_leading_eigenvector['country_code'] = df_leading_eigenvector['codes'].apply(lambda x: x[0])
df_leading_eigenvector['continent_code'] = df_leading_eigenvector['codes'].apply(lambda x: x[1])
df_leading_eigenvector
#df_walktrap
#df_infomap

,Unnamed: 0,name,community,codes,country_code,continent_code
0,0,Albania,0,"(AL, EU)",AL,EU
1,1,Bahamas,0,"(BS, NA)",BS,NA
2,2,Algeria,1,"(DZ, AF)",DZ,AF
3,3,British Virgin Islands,1,"(VG, NA)",VG,NA
4,4,United Kingdom,1,"(GB, EU)",GB,EU
...,...,...,...,...,...,...
190,190,Venezuela,0,"(VE, SA)",VE,SA
191,191,Vietnam,1,"(VN, AS)",VN,AS
192,192,Yemen,0,"(YE, AS)",YE,AS
193,193,Zambia,1,"(ZM, AF)",ZM,AF


In [7]:
df_countries = df_spinglass[['name','country_code']]
df_countries

,name,country_code
0,Albania,AL
1,Bahamas,BS
2,Algeria,DZ
3,British Virgin Islands,VG
4,United Kingdom,GB
...,...,...
190,Venezuela,VE
191,Vietnam,VN
192,Yemen,YE
193,Zambia,ZM


In [8]:
def geolocate(country,country_code):
    geolocator = Nominatim(user_agent = country)
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country, country_codes = country_code)
        # And return latitude and longitude
        return (round(loc.latitude,2), round(loc.longitude,2))
    except:
        # Return missing value
        loc = geolocator.geocode(country, country_codes = None)
        return (round(loc.latitude,2), round(loc.longitude,2))

In [9]:
df_countries['Lat_Long'] = df_countries.apply(lambda x: geolocate(x['name'], x['country_code']),axis = 1)

<ipython-input-9-a7c0add9ac3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['Lat_Long'] = df_countries.apply(lambda x: geolocate(x['name'], x['country_code']),axis = 1)


In [10]:
df_countries['Latitude'] = df_countries['Lat_Long'].apply(lambda x: x[0])
df_countries['Longitude'] = df_countries['Lat_Long'].apply(lambda x: x[1])

df_spinglass = df_spinglass.merge(df_countries, how = 'left')
df_walktrap = df_walktrap.merge(df_countries, how = 'left')
df_leading_eigenvector = df_leading_eigenvector.merge(df_countries, how = 'left')
df_walktrap

<ipython-input-10-d35070765225>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['Latitude'] = df_countries['Lat_Long'].apply(lambda x: x[0])
<ipython-input-10-d35070765225>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries['Longitude'] = df_countries['Lat_Long'].apply(lambda x: x[1])


,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,0,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,1,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
2,2,Algeria,2,"(DZ, AF)",DZ,AF,"(28.0, 3.0)",28.00,3.00
3,3,British Virgin Islands,1,"(VG, NA)",VG,NA,"(18.4, -64.57)",18.40,-64.57
4,4,United Kingdom,1,"(GB, EU)",GB,EU,"(54.7, -3.28)",54.70,-3.28
...,...,...,...,...,...,...,...,...,...
190,190,Venezuela,1,"(VE, SA)",VE,SA,"(8.0, -66.11)",8.00,-66.11
191,191,Vietnam,1,"(VN, AS)",VN,AS,"(13.29, 108.43)",13.29,108.43
192,192,Yemen,63,"(YE, AS)",YE,AS,"(16.35, 47.89)",16.35,47.89
193,193,Zambia,64,"(ZM, AF)",ZM,AF,"(-14.52, 27.56)",-14.52,27.56


In [11]:
df_spinglass.dropna(inplace = True)
df_spinglass.reset_index(drop = True, inplace = True)

df_walktrap.dropna(inplace = True)
df_walktrap.reset_index(drop = True, inplace = True)

df_leading_eigenvector.dropna(inplace = True)
df_leading_eigenvector.reset_index(drop = True, inplace = True)
df_spinglass

,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,2,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,2,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
2,2,Algeria,4,"(DZ, AF)",DZ,AF,"(28.0, 3.0)",28.00,3.00
3,3,British Virgin Islands,5,"(VG, NA)",VG,NA,"(18.4, -64.57)",18.40,-64.57
4,4,United Kingdom,4,"(GB, EU)",GB,EU,"(54.7, -3.28)",54.70,-3.28
...,...,...,...,...,...,...,...,...,...
190,190,Venezuela,1,"(VE, SA)",VE,SA,"(8.0, -66.11)",8.00,-66.11
191,191,Vietnam,5,"(VN, AS)",VN,AS,"(13.29, 108.43)",13.29,108.43
192,192,Yemen,1,"(YE, AS)",YE,AS,"(16.35, 47.89)",16.35,47.89
193,193,Zambia,5,"(ZM, AF)",ZM,AF,"(-14.52, 27.56)",-14.52,27.56


In [12]:
df_spinglass['Latitude'] = df_spinglass['Lat_Long'].apply(lambda x: x[0])
df_spinglass['Longitude'] = df_spinglass['Lat_Long'].apply(lambda x: x[1])

df_walktrap['Latitude'] = df_walktrap['Lat_Long'].apply(lambda x: x[0])
df_walktrap['Longitude'] = df_walktrap['Lat_Long'].apply(lambda x: x[1])

df_leading_eigenvector['Latitude'] = df_leading_eigenvector['Lat_Long'].apply(lambda x: x[0])
df_leading_eigenvector['Longitude'] = df_leading_eigenvector['Lat_Long'].apply(lambda x: x[1])
df_spinglass

,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,2,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,2,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
2,2,Algeria,4,"(DZ, AF)",DZ,AF,"(28.0, 3.0)",28.00,3.00
3,3,British Virgin Islands,5,"(VG, NA)",VG,NA,"(18.4, -64.57)",18.40,-64.57
4,4,United Kingdom,4,"(GB, EU)",GB,EU,"(54.7, -3.28)",54.70,-3.28
...,...,...,...,...,...,...,...,...,...
190,190,Venezuela,1,"(VE, SA)",VE,SA,"(8.0, -66.11)",8.00,-66.11
191,191,Vietnam,5,"(VN, AS)",VN,AS,"(13.29, 108.43)",13.29,108.43
192,192,Yemen,1,"(YE, AS)",YE,AS,"(16.35, 47.89)",16.35,47.89
193,193,Zambia,5,"(ZM, AF)",ZM,AF,"(-14.52, 27.56)",-14.52,27.56


In [13]:
df_spinglass['community'].value_counts()

5    71
4    41
1    39
3    22
2    18
6     2
0     2
Name: community, dtype: int64

In [14]:
df_walktrap['community'].value_counts()

1     124
4       7
65      1
24      1
19      1
     ... 
38      1
37      1
36      1
35      1
0       1
Name: community, Length: 66, dtype: int64

In [15]:
df_leading_eigenvector['community'].value_counts()

1    117
0     58
2     13
5      2
8      1
7      1
6      1
4      1
3      1
Name: community, dtype: int64

## Let's map the community for Spinglass

In [16]:
#Setting up the world countries data URL
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [17]:
#empty map
world_map_spinglass = folium.Map(tiles="cartodbpositron")
marker_cluster_spinglass = MarkerCluster().add_to(world_map_spinglass)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_spinglass)):
        lat = df_spinglass.iloc[i]['Latitude']
        long = df_spinglass.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_spinglass.iloc[i]['name'],
                                   df_spinglass.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_spinglass)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Spinglass Community Choropleth',
    data=df_spinglass,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_spinglass)
world_map_spinglass


In [18]:
world_map_spinglass.save('world_map_spinglass_no_inter.html')

## Walktrap

In [19]:
#empty map
world_map_walktrap = folium.Map(tiles="cartodbpositron")
marker_cluster_walktrap = MarkerCluster().add_to(world_map_walktrap)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_walktrap)):
        lat = df_walktrap.iloc[i]['Latitude']
        long = df_walktrap.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_walktrap.iloc[i]['name'],
                                   df_walktrap.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_walktrap)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Walktrap Community Choropleth',
    data=df_walktrap,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set1',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_walktrap)
world_map_walktrap

In [20]:
world_map_walktrap.save('world_map_walktrap_no_inter.html')

## Eigenvector

In [21]:
#empty map
world_map_eigen = folium.Map(tiles="cartodbpositron")
marker_cluster_eigen = MarkerCluster().add_to(world_map_eigen)
#for each coordinate, create circlemarker of user percent
for i in range(len(df_walktrap)):
        lat = df_leading_eigenvector.iloc[i]['Latitude']
        long = df_leading_eigenvector.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    Community : {}<br>"""
        popup_text = popup_text.format(df_leading_eigenvector.iloc[i]['name'],
                                   df_leading_eigenvector.iloc[i]['community']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster_eigen)

folium.Choropleth(
    #The GeoJSON data to represent the world country
    geo_data=country_shapes,
    name='Eigen Community Choropleth',
    data=df_leading_eigenvector,
    #The column aceppting list with 2 value; The country name and  the numerical value
    columns=['name',"community"],
    key_on='feature.properties.name',
    fill_color='Set2',
    nan_fill_color='white',
    legend_name = 'Community'
).add_to(world_map_eigen)
world_map_eigen

In [22]:
df_leading_eigenvector[df_leading_eigenvector['community'] == 0]

,Unnamed: 0,name,community,codes,country_code,continent_code,Lat_Long,Latitude,Longitude
0,0,Albania,0,"(AL, EU)",AL,EU,"(41.0, 20.0)",41.00,20.00
1,1,Bahamas,0,"(BS, NA)",BS,NA,"(24.77, -78.0)",24.77,-78.00
6,6,Andorra,0,"(AD, EU)",AD,EU,"(42.54, 1.57)",42.54,1.57
8,8,Costa Rica,0,"(CR, NA)",CR,NA,"(10.27, -84.07)",10.27,-84.07
12,12,Nevada,0,"(Unknown, Unknown)",Unknown,Unknown,"(39.52, -116.85)",39.52,-116.85
13,13,Niue,0,"(NU, OC)",NU,OC,"(-19.05, -169.86)",-19.05,-169.86
14,14,Panama,0,"(PA, NA)",PA,NA,"(8.56, -81.13)",8.56,-81.13
17,17,Seychelles,0,"(SC, AF)",SC,AF,"(-4.66, 55.45)",-4.66,55.45
18,18,Uruguay,0,"(UY, SA)",UY,SA,"(-32.88, -56.02)",-32.88,-56.02
26,26,Argentina,0,"(AR, SA)",AR,SA,"(-35.0, -64.97)",-35.00,-64.97


In [23]:
world_map_eigenvector.save('world_map_eigenvector_no_inter.html')

NameError: name 'world_map_eigenvector' is not defined